In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingRegressor, RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import lime
import lime.lime_tabular

# Load the Excel file
file_path = "/content/10-Java_AST_in_.xlsx"
xls = pd.ExcelFile(file_path)
df = pd.read_excel(xls, sheet_name="in")

# Display dataset info
print("Dataset Information:\n")
print(df.info())
print("\nFirst 5 Rows:\n")
print(df.head())

# Define features (X) and target (y)
X = df.drop(columns=['Final_Marks', 'error_count'])
y = df['Final_Marks']  #

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define base regressors
base_models = [
    ('ridge', Ridge()),
    ('dt', DecisionTreeRegressor()),
    ('rf', RandomForestRegressor(n_estimators=50, random_state=42)),
    ('svr', SVR())
]

# Define Stacking Regressor with Ridge as the final estimator
stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=Ridge())

# Train the model
stacking_regressor.fit(X_train, y_train)


train_score = stacking_regressor.score(X_train, y_train)
test_score = stacking_regressor.score(X_test, y_test)

print(f"\nStacking Regressor Train Score: {train_score:.4f}")
print(f"Stacking Regressor Test Score: {test_score:.4f}")

# Create a pipeline with scaling and stacking regressor
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('stacking', stacking_regressor)
])

# Train the pipeline
pipeline.fit(X_train, y_train)

# Evaluate the pipeline
train_score_pipeline = pipeline.score(X_train, y_train)
test_score_pipeline = pipeline.score(X_test, y_test)

print(f"\nPipeline Train Score: {train_score_pipeline:.4f}")
print(f"Pipeline Test Score: {test_score_pipeline:.4f}")

# Initialize LIME explainer
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=X.columns.tolist(),
    mode="regression"
)

# Explain a single prediction (first instance in the test set)
idx = 0  # Example index
exp = explainer.explain_instance(X_test.iloc[idx].values, pipeline.predict)
lime_explanation = exp.as_list()

# Print LIME explanation
print("\nLIME Explanation (Feature Importance for a Single Prediction):\n")
for feature, importance in lime_explanation:
    print(f"{feature}: {importance:.4f}")

# (Optional) Visualize LIME explanation
exp.show_in_notebook()


Dataset Information:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1232 entries, 0 to 1231
Columns: 770 entries, ast_0 to error_count
dtypes: float64(768), int64(2)
memory usage: 7.2 MB
None

First 5 Rows:

      ast_0     ast_1     ast_2     ast_3     ast_4     ast_5     ast_6  \
0 -0.353538 -0.500395  0.496051 -1.184219 -0.201379  1.553561  0.828044   
1 -0.035058 -1.062878  0.585158 -2.126432  0.251542  1.598461  1.259976   
2 -0.199280  0.203032 -0.678487 -1.879159 -0.453607  1.100767  1.252518   
3  0.556859  0.321459 -0.201903 -1.736377 -0.738745  0.965423  1.116394   
4 -0.442604 -0.492590  1.104117 -2.577588  0.909446  2.396572  1.334725   

      ast_7     ast_8     ast_9  ...   ast_760   ast_761   ast_762   ast_763  \
0  0.826938 -1.464319  2.654585  ...  4.175181  0.107633 -1.740333 -1.813466   
1  2.040154 -2.605130  3.339965  ...  3.496885 -0.300682 -1.583440 -0.402666   
2  1.052316 -2.092935  2.634723  ...  2.395128 -0.140103 -1.182756 -2.031094   
3  1.227975 -2.07

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



LIME Explanation (Feature Importance for a Single Prediction):

ast_118 > 2.13: -0.3747
ast_194 > 2.20: 0.3664
ast_488 <= -5.15: 0.3635
ast_336 <= -3.50: -0.3564
ast_80 > 1.95: 0.3560
ast_473 > 2.89: 0.3164
ast_322 <= -1.08: -0.3145
ast_193 > 0.69: -0.3136
ast_470 > -1.48: -0.2636
ast_707 <= 2.25: -0.2215
